In [1]:
import feather
import scipy as sp
import numpy as np
import pandas as pd
import xgboost as xgb

from collections import Counter
from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold, GroupKFold

import matplotlib.pyplot as plt
import seaborn as sns

def get_score(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def get_y():
    return pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv', usecols=[target]).values.flatten()
    
def run_xgb_model(X_train, y_train, X_valid, y_valid,
            categorical_features, numerical_features,
            predictors, maxvalue_dict, fold_id):
    d_train = xgb.DMatrix(data=X_train, label=y_train, feature_names=predictors)
    d_valid = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=predictors)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(dtrain=d_train, evals=watchlist, params=MODEL_PARAMS_XGB, **FIT_PARAMS)
    
    # validation score
    y_pred_valid = model.predict(d_valid, ntree_limit=model.best_ntree_limit)
    
    return y_pred_valid
 
def plot_mean_feature_importances(feature_importances, max_num=50, importance_type='gain', path=None):
    mean_gain = feature_importances[[importance_type, 'feature']].groupby('feature').mean()
    feature_importances['mean_' + importance_type] = feature_importances['feature'].map(mean_gain[importance_type])

    if path is not None:
        data = feature_importances.sort_values('mean_'+importance_type, ascending=False).iloc[:max_num, :]
        plt.clf()
        plt.figure(figsize=(16, 8))
        sns.barplot(x=importance_type, y='feature', data=data)
        plt.tight_layout()
        plt.savefig(path)
    
    return feature_importances

def to_bins(x, borders):
    for i in range(len(borders)):
        if x <= borders[i]:
            return i
    return len(borders)

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _loss(self, coef, X, y, idx):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        ll = -get_score(y, X_p)
        return ll

    def fit(self, X, y):
        coef = [1.5, 2.0, 2.5, 3.0]
        golden1 = 0.618
        golden2 = 1 - golden1
        ab_start = [(1, 2), (1.5, 2.5), (2, 3), (2.5, 3.5)]
        for it1 in range(10):
            for idx in range(4):
                # golden section search
                a, b = ab_start[idx]
                # calc losses
                coef[idx] = a
                la = self._loss(coef, X, y, idx)
                coef[idx] = b
                lb = self._loss(coef, X, y, idx)
                for it in range(20):
                    # choose value
                    if la > lb:
                        a = b - (b - a) * golden1
                        coef[idx] = a
                        la = self._loss(coef, X, y, idx)
                    else:
                        b = b - (b - a) * golden2
                        coef[idx] = b
                        lb = self._loss(coef, X, y, idx)
        self.coef_ = {'x': coef}

    def predict(self, X, coef):
        X_p = np.array([to_bins(pred, coef) for pred in X])
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [2]:
target = 'AdoptionSpeed'
len_train = 14993
len_test = 3948
    
    
# ===============
# Params
# ===============
seed = 777
n_splits = 5
np.random.seed(seed)

# feature engineering
n_components = 5
img_size = 256
batch_size = 256

# model
MODEL_PARAMS_XGB = {
    'eval_metric': 'rmse',
    'seed': 1337,
    'eta': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.85,
    'tree_method': 'gpu_hist',
    'device': 'gpu',
    'silent': 1,
}
FIT_PARAMS = {
    'num_boost_round': 5000,
    'early_stopping_rounds': 100,
    'verbose_eval': 10000,
}

# define
maxvalue_dict = {}
categorical_features = [
     'Breed1',
     'Breed2',
     'Color1',
     'Color2',
     'Color3',
     'Dewormed',
     'FurLength',
     'Gender',
     'Health',
     'MaturitySize',
     'State',
     'Sterilized',
     'Type',
     'Vaccinated',
     'Type_main_breed',
     'BreedName_main_breed',
     'Type_second_breed',
     'BreedName_second_breed',
]
numerical_features = []
text_features = ['Name', 'Description']
remove = ['index', 'seq_text', 'PetID', 'Name', 'Description', 'RescuerID', 'StateName', 'annots_top_desc']

In [3]:
train = feather.read_dataframe('X_train460.feather')
n_train = len(train)

In [ ]:
{
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'num_leaves': 63,
    'subsample': 0.9,
    'subsample_freq': 1,
    'colsample_bytree': 0.6,
    'max_depth': 9,
    'max_bin': 127,
    'reg_alpha': 0.11,
    'reg_lambda': 0.01,
    'min_child_weight': 0.2,
    'min_child_samples': 20,
    'min_gain_to_split': 0.02,
    'min_data_in_bin': 3,
    'bin_construct_sample_cnt': 5000,
    'cat_l2': 10,
    'verbose': -1,
    'nthread': -1,
    'seed': 777,
}
bounds = [{'name': 'alpha', 'type': 'continuous', 'domain': (0, 15)},
          {'name': 'lambda', 'type': 'continuous', 'domain': (0, 15)},
          {'name': 'max_depth', 'type': 'discrete', 'domain': (5, 7, 10, 12, 15)},
          {'name': 'max_leaves', 'type': 'discrete', 'domain': (10, 20, 31, 63, 127, 255)},
          {'name': 'colsample_bylevel', 'type': 'continuous', 'domain': (0.01, 1.0)},
          {'name': 'min_samples_leaf', 'type': 'discrete', 'domain': (10, 20, 30, 50, 60, 80, 100, 120, 150)},
          ]

In [ ]:
%%time
MODEL_PARAMS_XGB = {
    'eval_metric': 'rmse',
    'seed': 1337,
    'eta': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.85,
    'max_leaves': 127,
    'alpha': 0.1,
    'tree_method': 'gpu_hist',
    'device': 'gpu',
    'silent': 1,
}
categorical_features = list(set(categorical_features) - set(remove))
numerical_features = list(set(train.columns) - set(categorical_features + [target] + remove))
predictors = categorical_features + numerical_features
predictors = [c for c in predictors if "gnvec" not in c and "glove_mag" not in c and "img_" not in c]
train = train.loc[:, ~train.columns.duplicated()]

X = train.loc[:, predictors]
y =  feather.read_dataframe('../input/X_train.feather')["AdoptionSpeed"].values
rescuer_id = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv').loc[:, 'RescuerID'].iloc[:len_train]

y_pred = np.empty(len_train,)
y_test = []

#cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1337)
#for fold_id, (train_index, valid_index) in enumerate(cv.split(range(len(X)), y)):
cv = GroupKFold(n_splits=n_splits)
for fold_id, (train_index, valid_index) in enumerate(cv.split(range(len(X)), y=None, groups=rescuer_id)): 
    X_train = X.loc[train_index, :]
    X_valid = X.loc[valid_index, :]
    y_train = y[train_index]
    y_valid = y[valid_index]

    y_pred_valid = run_xgb_model(X_train, y_train, X_valid, y_valid,
                     categorical_features, numerical_features,
                     predictors, maxvalue_dict, fold_id)
    y_pred[valid_index] = y_pred_valid.ravel()


optR = OptimizedRounder()
optR.fit(y_pred, y)
coefficients = optR.coefficients()
y_pred_opt = optR.predict(y_pred, coefficients)
score = get_score(y, y_pred_opt)
print(score)

[0]	train-rmse:2.32031	valid-rmse:2.30027
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[953]	train-rmse:0.708063	valid-rmse:1.04865

[0]	train-rmse:2.30251	valid-rmse:2.37194
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[910]	train-rmse:0.723786	valid-rmse:1.1169

[0]	train-rmse:2.32649	valid-rmse:2.27468
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[461]	train-rmse:0.830407	valid-rmse:1.06378

[0]	train-rmse:2.3286	valid-rmse:2.26587
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[780]	tr

In [12]:
print(score)

0.40797404734999587


In [9]:
0.42707925650676515

0.42707925650676515